In [2]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 41.3 MB/s eta 0:00:00


In [3]:
# -*- coding: utf-8 -*-
"""
CNN for CIFAR-100 — thesis-oriented baseline.
Target: ~60–65% test accuracy without transfer learning.
"""

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import optim, nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchmetrics

In [4]:


# ---------------------------------------------------------------------------
# Data — CIFAR-100 correct normalization + augmentation (train only)
# ---------------------------------------------------------------------------
# Per-channel normalization for CIFAR-100 (mandatory)
CIFAR100_MEAN = (0.5071, 0.4867, 0.4408)
CIFAR100_STD = (0.2675, 0.2565, 0.2761)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),   # 32→36 padded then crop 32
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR100_MEAN, CIFAR100_STD),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR100_MEAN, CIFAR100_STD),
])

batch_size = 128
print("Loading CIFAR-100 dataset...")
train_dataset = datasets.CIFAR100(root="dataset/", download=True, train=True, transform=transform_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

test_dataset = datasets.CIFAR100(root="dataset/", download=True, train=False, transform=transform_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset)}, Test samples: {len(test_dataset)}, Classes: 100")


def imshow(img):
    """Display image tensor (denormalized for visualization)."""
    img = img * torch.tensor(CIFAR100_STD).view(3, 1, 1) + torch.tensor(CIFAR100_MEAN).view(3, 1, 1)
    img = torch.clamp(img, 0, 1)
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)))
    plt.show()


Loading CIFAR-100 dataset...


100%|██████████| 169M/169M [00:09<00:00, 18.1MB/s]


Training samples: 50000, Test samples: 10000, Classes: 100


In [5]:
# ---------------------------------------------------------------------------
# CNN: 8 conv layers in 4 blocks — [Conv→BatchNorm→ReLU]×2 → MaxPool → Dropout
# Block 1: 32 filters, Block 2: 64, Block 3: 128, Block 4: 256 → GAP → Dense(100)
# ---------------------------------------------------------------------------
class CNN(nn.Module):
    def __init__(self, num_classes=100):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1: 2 Conv (32 filters)
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            # Block 2: 2 Conv (64 filters)
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            # Block 3: 2 Conv (128 filters)
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.3),
            # Block 4: 2 Conv (256 filters)
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



In [7]:
# ---------------------------------------------------------------------------
# Training: SGD + momentum, weight decay, cosine LR, 150–200 epochs
# ---------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = CNN(num_classes=100).to(device)
print("\nModel architecture:")
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
num_epochs = 50


scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

print(f"\nStarting training for {num_epochs} epochs (SGD lr=0.1, momentum=0.9, weight_decay=1e-4, CosineAnnealing)...")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    for data, targets in pbar:
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        scores = model(data)
        loss = criterion(scores, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")
    scheduler.step()
    avg_loss = running_loss / len(train_loader)
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] avg_loss: {avg_loss:.4f} lr: {scheduler.get_last_lr()[0]:.6f}")

print("Training completed!")


Using device: cpu

Model architecture:
CNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.25, inplace=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2

Epoch [1/50]: 100%|██████████| 391/391 [07:39<00:00,  1.18s/it, loss=4.1658]


Epoch [1/50] avg_loss: 4.2083 lr: 0.099901


Epoch [10/50]: 100%|██████████| 391/391 [07:33<00:00,  1.16s/it, loss=2.5893]


Epoch [10/50] avg_loss: 2.9940 lr: 0.090451


...

Epoch [30/50] avg_loss: 2.1127 lr: 0.034549


Epoch [40/50]: 100%|██████████| 391/391 [07:33<00:00,  1.16s/it, loss=1.7326]


Epoch [40/50] avg_loss: 1.8671 lr: 0.009549


Epoch [50/50]: 100%|██████████| 391/391 [07:38<00:00,  1.17s/it, loss=2.0763]

Epoch [50/50] avg_loss: 1.7731 lr: 0.000000
Training completed!


In [8]:
# ---------------------------------------------------------------------------
# Evaluation
# ---------------------------------------------------------------------------
print("\nEvaluating on test set...")
acc_metric = torchmetrics.Accuracy(task="multiclass", num_classes=100)
precision_metric = torchmetrics.Precision(task="multiclass", num_classes=100, average="macro")
recall_metric = torchmetrics.Recall(task="multiclass", num_classes=100, average="macro")

model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        acc_metric(preds.cpu(), labels.cpu())
        precision_metric(preds.cpu(), labels.cpu())
        recall_metric(preds.cpu(), labels.cpu())

test_accuracy = acc_metric.compute()
test_precision = precision_metric.compute()
test_recall = recall_metric.compute()

print(f"\nTest Results:")
print(f"  Test Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"  Test Precision: {test_precision:.4f}")
print(f"  Test Recall:    {test_recall:.4f}")
print("\n" + "="*60)
print("CNN Training and Evaluation Complete!")
print("="*60)



Evaluating on test set...


Evaluating: 100%|██████████| 79/79 [00:37<00:00,  2.13it/s]


Test Results:
  Test Accuracy:  0.5752 (57.52%)
  Test Precision: 0.5849
  Test Recall:    0.5752

CNN Training and Evaluation Complete!
